# Jupyter Notebook to clean install all the contracts


### Necessary packages 

In [57]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Set Up Environment

In [58]:
network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom
contracts_file_custom = 'contracts_20201109050405.pkl'
#contracts_20201109120425

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.237.205.52:9000/",       "nid": 3},
"local":   {"iconservice": "http://localhost:9000/",           "nid": 3}
}

env = connections[network]

In [59]:
class AutoVivification(dict):
    """Implementation of perl's autovivification feature."""
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value

In [60]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
from shutil import make_archive
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass


ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"

with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result




### Required service

In [61]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

# Importing wallet

### Importing from private key

In [45]:

private="a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a"
deployer_wallet = KeyWallet.load(bytes.fromhex(private))
deployer_wallet.get_address()


'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'

In [56]:
icon_service.get_balance("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce")

ConnectionError: HTTPSConnectionPool(host='bicon.net.solidwallet.io', port=443): Max retries exceeded with url: /api/v3 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa007a41dc0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

### Importing wallet from keystore file

In [87]:
# Set to the address of the keystore file and give the password when prompted
password = getpass()
deployer_wallet = KeyWallet.load("./keystores/keystore_test1.json",password)
deployer_wallet.get_address()
icon_service.get_balance(deployer_wallet.get_address())

ConnectionError: HTTPConnectionPool(host='localhost', port=9000): Max retries exceeded with url: /api/v3 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fb022479fd0>: Failed to establish a new connection: [Errno 111] Connection refused'))

### Creating new wallet

In [ ]:
deployer_wallet = KeyWallet.create()
print(deployer_wallet.get_address())
caller_wallet = KeyWallet.create()
print(caller_wallet.get_address())

In [50]:
# Add the contracts. Copy the output from this cell the the one above to save the contract addresses.
contracts=AutoVivification()

deploy = ['addressProvider', 'feeProvider', 'lendingPool', 'lendingPoolCore', 'lendingPoolDataProvider','oToken','priceOracle','sample_token']
# for directory in {"score"}:
directory=os.getcwd()
with os.scandir(directory) as it:
    for file in it:
        archive_name = directory + "/" + file.name
        if file.is_dir() and file.name in deploy:
            make_archive(archive_name, "zip", directory, file.name)
            contracts[file.name]['zip'] = archive_name + '.zip'


contracts

{'lendingPoolDataProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider.zip'},
 'lendingPool': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool.zip'},
 'feeProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/feeProvider.zip'},
 'lendingPoolCore': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolCore.zip'},
 'oToken': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/oToken.zip'},
 'priceOracle': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/priceOracle.zip'},
 'sample_token': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/sample_token.zip'},
 'addressProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/addressProvider.zip'}}

In [ ]:
# Deploying contracts


In [55]:
params = {}
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('addressProvider'))\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)
res = get_tx_result(tx_hash)
contracts[score]["SCORE"] = res.get('scoreAddress', '')
now = datetime.utcnow()
filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)


ConnectionError: HTTPSConnectionPool(host='bicon.net.solidwallet.io', port=443): Max retries exceeded with url: /api/debug/v3 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fa007aa1250>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

In [51]:

# contracts['sample_token']['SCORE']="cx19be1be7b4e7750863edf966faac3f42dec21025"
# contracts['oToken']['SCORE']="cx4258464e395b3ef3b73a146947eb41393e8c5873"
# contracts['lendingPool']['SCORE']="cx35b12ca4e9c672518db3c06d92ed60d50e7d5554"
# contracts['lendingPoolCore']['SCORE']="cx327a386a9f8076045679fc132a8baa9e270ca43b"
# contracts['lendingPoolDataProvider']['SCORE']="cxa198eaf0a589adf0ab31b523d1f8912f47f6f293"
# contracts['priceOracle']['SCORE']="cx3f6477bf290eb91257b806ece614ab18b1c7be41"
# contracts['addressProvider']['SCORE']="cxed440f1da2cf3087ac5ef12217a13f6321de0375"
# contracts['feeProvider']['SCORE']="cxede316aa65b7df13633599e67490daf1fd32b8a5"
pprint(contracts)
# now = datetime.utcnow()
# filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
# print(f'saving contracts list as {filename}')
# with open(filename, 'wb') as f:
#     pkl.dump(contracts, f)

{'addressProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/addressProvider.zip'},
 'feeProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/feeProvider.zip'},
 'lendingPool': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool.zip'},
 'lendingPoolCore': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolCore.zip'},
 'lendingPoolDataProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider.zip'},
 'oToken': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/oToken.zip'},
 'priceOracle': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/priceOracle.zip'},
 'sample_token': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/sample_token.zip'}}


In [62]:
for score, loc in contracts.items():
    params = {}
    print(score,loc)
    if score == "sample_token":
        params = {'_initialSupply':500000000, '_decimals': 18}
    elif score == "oToken":
        params = {"_name":"BridgeUSDInterestToken","_symbol":"oUSDb"}
    deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(loc["zip"]))\
        .params(params)\
        .build()

    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[score]["SCORE"] = res.get('scoreAddress', '')
now = datetime.utcnow()
filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)
contracts

lendingPoolDataProvider {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider.zip', 'SCORE': 'cxa198eaf0a589adf0ab31b523d1f8912f47f6f293'}
lendingPool {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool.zip', 'SCORE': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
feeProvider {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/feeProvider.zip', 'SCORE': 'cxede316aa65b7df13633599e67490daf1fd32b8a5'}
lendingPoolCore {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolCore.zip', 'SCORE': 'cx327a386a9f8076045679fc132a8baa9e270ca43b'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending tra

{'lendingPoolDataProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider.zip',
  'SCORE': 'cx9f87296aca0d3b1ca6c2471181db61090b3b3181'},
 'lendingPool': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool.zip',
  'SCORE': 'cx9a1e70f60ee138d8e2c2eee0daf28a7d0645bd02'},
 'feeProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/feeProvider.zip',
  'SCORE': 'cx0735789bb3a43f5ed03831add95891fbb0681ef0'},
 'lendingPoolCore': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolCore.zip',
  'SCORE': 'cx879e276135a6f25236a8abf2e4c9801887538f60'},
 'oToken': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/oToken.zip',
  'SCORE': 'cx2f9c6ae40e6ae4c8d39961b3f3d701363b361773'},
 'priceOracle': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/priceOracle.zip',
  'SCORE': 'cx465f9f302bb8bf1a66a2280017a13c526e88762c'},
 'sample_token': {'zip': '/home/shubham/Work/ICON/openmoneymarket-

In [63]:
pprint(contracts)

{'addressProvider': {'SCORE': 'cxd7b6cf3e486f7a4b462d25c918fff0026551bef0',
                     'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/addressProvider.zip'},
 'feeProvider': {'SCORE': 'cx0735789bb3a43f5ed03831add95891fbb0681ef0',
                 'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/feeProvider.zip'},
 'lendingPool': {'SCORE': 'cx9a1e70f60ee138d8e2c2eee0daf28a7d0645bd02',
                 'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool.zip'},
 'lendingPoolCore': {'SCORE': 'cx879e276135a6f25236a8abf2e4c9801887538f60',
                     'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolCore.zip'},
 'lendingPoolDataProvider': {'SCORE': 'cx9f87296aca0d3b1ca6c2471181db61090b3b3181',
                             'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider.zip'},
 'oToken': {'SCORE': 'cx2f9c6ae40e6ae4c8d39961b3f3d701363b361773',
            'zip': '/home/shubham/Work

# Updating a contract

In [36]:
contract = 'lendingPool'
update = contracts[contract]['SCORE']
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contracts[contract]["zip"]))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0xa77c8cd5457c93c267a472e685a67825802efaef9e4902dd77be6ea7b0a29c5b',
 'blockHeight': 96428,
 'blockHash': '0xd805acfe7df5115318a02ea7e339dd6fcb55ff378577fc921aeaebc1d622876b',
 'txIndex': 1,
 'to': 'cxad29953f5b8957704e9092632bccc3b1aac62eea',
 'scoreAddress': 'cxad29953f5b8957704e9092632bccc3b1aac62eea',
 'stepUsed': 1635708000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1635708000,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

# Configuring SCOREs's parameters

In [64]:
settings = [{'contract': 'lendingPool', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']['SCORE']}},
            {'contract': 'lendingPool', 'method': 'setUSDbAddress', 'params':{'_address': contracts['sample_token']['SCORE']}},
            {'contract': 'lendingPool', 'method': 'setDataProvider', 'params':{'_address': contracts['lendingPoolDataProvider']['SCORE']}},
            {'contract': 'lendingPool', 'method': 'setFeeProvider', 'params':{'_address': contracts['feeProvider']['SCORE']}},
            {'contract': 'feeProvider', 'method': 'setLoanOriginationFeePercentage', 'params':{'_percentage': 25*10**14}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['sample_token']['SCORE'],'_sym':"USDb"}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']['SCORE']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setOracleAddress', 'params':{'_address': contracts['priceOracle']['SCORE']}},
            {'contract': 'oToken', 'method': 'setCoreAddress', 'params':{'_address':contracts['lendingPoolCore']['SCORE']}},
            {'contract': 'oToken', 'method': 'setReserveAddress', 'params':{'_address':contracts['sample_token']['SCORE']}},
            {'contract': 'oToken', 'method': 'setDataProviderAddress', 'params':{'_address':contracts['lendingPoolDataProvider']['SCORE']}},
            {'contract': 'oToken', 'method': 'setLendingPoolAddress', 'params':{'_address':contracts['lendingPool']['SCORE']}},
            {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'USDb','_quote':'USD','_rate':5*10**17}},
            {'contract': 'addressProvider', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPoolCore']['SCORE']}},
            {'contract': 'addressProvider', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']['SCORE']}},
            {'contract': 'addressProvider', 'method': 'setUSDb', 'params':{'_address':contracts['sample_token']['SCORE']}},
            {'contract': 'addressProvider', 'method': 'setoUSDb', 'params':{'_address':contracts['oToken']['SCORE']}},
            {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['sample_token']['SCORE'],"optimalUtilizationRate":f"6{'0'*17}","baseBorrowRate":"0","slopeRate1":f"7{'0'*16}","slopeRate2":f"3{'0'*18}"} ]}}]

In [86]:
params={"_constants": [{"reserve":contracts['sample_token']['SCORE'],"optimalUtilizationRate":f"6{'0'*17}","baseBorrowRate":f"1{'0'*16}","slopeRate1":f"4{'0'*16}","slopeRate2":f"5{'0'*17}"} ]}

transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts['lendingPoolCore']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method('setReserveConstants')\
        .params(params)\
        .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

{'txHash': '0x3606508078f11c4d36b57cbb82dffebc248b1c594a836ad32859b8ffb505d8e7', 'blockHeight': 444320, 'blockHash': '0x6e4d07a371b61375acc41635aaa2fd11626a6218ff8003778604b7a4e41ad188', 'txIndex': 1, 'to': 'cx879e276135a6f25236a8abf2e4c9801887538f60', 'stepUsed': 180480, 'stepPrice': 10000000000, 'cumulativeStepUsed': 180480, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [65]:
for sett in settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']]['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))

96aca0d3b1ca6c2471181db61090b3b3181', 'stepUsed': 152720, 'stepPrice': 10000000000, 'cumulativeStepUsed': 152720, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

## Adding new reserve in LendingPoolCore



In [66]:
params ={"_reserve": {"reserveAddress":contracts['sample_token']['SCORE'],"oTokenAddress":contracts['oToken']['SCORE'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"60000000000000000000","liquidationThreshold":"65","liquidationBonus":"10","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'txHash': '0xd608ccfac9255fe236e0240e07168545e838da1c90db78bef90113cc445d2c5d',
 'blockHeight': 443556,
 'blockHash': '0xfa57516b8a1dfc13ca2b3f9752a83036e69e793418b3e5a343935ab3e9f2de50',
 'txIndex': 1,
 'to': 'cx879e276135a6f25236a8abf2e4c9801887538f60',
 'stepUsed': 266880,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 266880,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

# Depositing USDb

In [67]:
data = "{\"method\": \"deposit\", \"params\": {\"amount\": 1000000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool']['SCORE'], "_value": 1000000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0xb1fc5062c7d5fa859a08dd5b505b322bd03450067aec12f06e456c79c25c7d26',
 'blockHeight': 443726,
 'blockHash': '0x1bcfa6746677749da2e16fb7b42ff3708e2dcb3a9fcb7f6a35458f58c72ea4e6',
 'txIndex': 1,
 'to': 'cx2c002610b8aeaff84f6cb97c28294b1986016143',
 'stepUsed': 729440,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 729440,
 'eventLogs': [{'scoreAddress': 'cx879e276135a6f25236a8abf2e4c9801887538f60',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cx2c002610b8aeaff84f6cb97c28294b1986016143',
    '0x0',
    '0x0'],
   'data': ['0xde0b6b3a7640000', '0xde0b6b3a7640000']},
  {'scoreAddress': 'cx2f9c6ae40e6ae4c8d39961b3f3d701363b361773',
   'indexed': ['Mint(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x0']},
  {'scoreAddress': 'cx2f9c6ae40e6ae4c8d39961b3f3d701363b361773',
   'indexed': ['Mint(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x3635c9adc5dea00000']},
  {'scoreAddress': 'cx2f9c6ae40e6ae4

# Borrowing USDb

In [75]:
params ={"_reserve": contracts['sample_token']['SCORE'], "_amount":150*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("borrow")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx9a1e70f60ee138d8e2c2eee0daf28a7d0645bd02', 'stepLimit': '0x11a350', 'timestamp': '0x5b3ab99c871a5', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'borrow', 'params': {'_reserve': 'cx2c002610b8aeaff84f6cb97c28294b1986016143', '_amount': '0x821ab0d4414980000'}}, 'signature': 'nnBAhBv6BGeWZE63Rr6d02x+c/R36yjf49WvR/G5VLd6hz7tKN6lL6lG/kghrjVg0ZPWIf2KdJi+rmBPoRrFWAA='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x4f24f998ab722bbcdd6589d23f1bd67cc98a4f30ff863710b291f60545017e0f',
 'blockHeight': 443982,
 'blockHash': '0x795f4716b2053eca2b1b8585c02c10fe30705843841760035bf91b80abb581c9',
 'txIndex': 1,
 'to': 'cx9a1e70f60ee138d8e2c2eee0daf28a7d0645bd02',
 'stepUsed': 1055920,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1055920,
 'eventLogs': [{'scoreAddress': 'cx879e276135a6f25236a8abf2e4c9801887538f60',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cx2c002610b8aeaff84f6cb97c28294b1986016143',
    '0x864ae65482800',
    '0x3e2c284391c000'],
   'data': ['0xde0b6b3a7640000', '0xde0b6b3a7640000']},
  {'scoreAddress': 'cx2c002610b8aeaff84f6cb97c28294b1986016143',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cx879e276135a6f25236a8abf2e4c9801887538f60',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce',
    '0x821ab0d4414980000'],
   'data': ['0x4e6f6e65']}],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x0

### Getting reserve data

In [88]:
params = {'_reserve': contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
# for key,values in response.items():
#     if values=="0x1" or values =="0x0" or "Address" in key:
#         continue
#     if key!="lastUpdateTimestamp":
#         response[key]=int(values,0)/10**18
#     else:
#         response[key]=int(values,0)

pprint(response)

{'availableLiquidity': '0x2e141ea081ca080000',
 'baseLTVasCollateral': '0x340aad21b3b700000',
 'borrowCumulativeIndex': '0xde0b6b3a7640000',
 'borrowRate': '0x3e2c284391c000',
 'borrowingEnabled': '0x1',
 'decimals': '0x12',
 'isActive': '0x1',
 'isFreezed': '0x0',
 'lastUpdateTimestamp': '0x5b3ab99def2bb',
 'liquidationBonus': '0xa',
 'liquidationThreshold': '0x41',
 'liquidityCumulativeIndex': '0xde0b6b3a7640000',
 'liquidityRate': '0x864ae65482800',
 'oTokenAddress': 'cx2f9c6ae40e6ae4c8d39961b3f3d701363b361773',
 'reserveAddress': 'cx2c002610b8aeaff84f6cb97c28294b1986016143',
 'totalBorrows': '0x821ab0d4414980000',
 'totalLiquidity': '0x3635c9adc5dea00000',
 'usageAsCollateralEnabled': '0x1'}


In [93]:
params = {'_reserve': contracts['sample_token']['SCORE'],'_availableLiquidity':850*10**18,'_totalBorrows':150*10**18}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']['SCORE'])\
    .method("calculateInterestRates")\
    .params(params)\
    .build()
response = icon_service.call(_call)
print(response)

{'borrowRate': '0x470de4df820000', 'liquidityRate': '0x997a2bce4c000'}


### Getting user account data

In [29]:
params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(LENDING_POOL_DATA_PROVIDER)\
    .method("getUserAccountData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'currentLiquidationThreshold': 2.5838e-14,
 'currentLtv': 60.0,
 'healthFactor': 203882.82366782034,
 'healthFactorBelowThreshold': 0.0,
 'totalBorrowBalanceUSD': 0.5000003816736971,
 'totalCollateralBalanceUSD': 397.5000009264478,
 'totalFeesUSD': 0.00375,
 'totalLiquidityBalanceUSD': 397.5000009264478}


### Getting user reserve data

In [94]:
params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' ,"_reserve":"cx072d8f95877a53df350e3dc3d8dba2f379037d42"}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(LENDING_POOL_DATA_PROVIDER)\
    .method("getUserReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

NameError: name 'LENDING_POOL_DATA_PROVIDER' is not defined

### Get reserve configuraitions


In [25]:
params = {"_reserve":"cx072d8f95877a53df350e3dc3d8dba2f379037d42"}
# params={"_base":"USDb","_quote":"USD"}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(LENDING_POOL_CORE)\
    .method("getReserveConfiguration")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    response[key]=int(values,0)
pprint(response)


{'baseLTVasCollateral': 60000000000000000000,
 'borrowingEnabled': 1,
 'decimals': 18,
 'isActive': 1,
 'liquidationBonus': 10,
 'liquidationThreshold': 65,
 'usageAsCollateralEnabled': 1}


In [17]:

params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(oUSDb)\
    .method("principalBalanceOf")\
    .params(params)\
    .build()
response = icon_service.call(_call)

print(int(response,0))

795000001589272371715


In [18]:

params = {'_owner': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(oUSDb)\
    .method("balanceOf")\
    .params(params)\
    .build()
response = icon_service.call(_call)
print(int(response,0))

795000001744001199280


In [ ]:
995000000570919367819
995000000564440311889
894000001574533095848
894000000615847612243
894000001574533095848
795000001589272371715
795000001731223349511